# Fruits CNN Export

In [1]:
import tensorflow as tf
import keras
from tensorflow.python.saved_model import signature_constants
from tensorflow.python.saved_model import tag_constants
from tensorflow.python.saved_model import builder
import os

Using TensorFlow backend.


## Load the model with Keras

In [2]:
fruits_cnn = keras.models.load_model('../fruits.h5')

## Build the fruits-cnn signature

In [3]:
tensor_info_input = tf.saved_model.utils.build_tensor_info(fruits_cnn.input)
tensor_info_output = tf.saved_model.utils.build_tensor_info(fruits_cnn.output)
prediction_signature = (
    tf.saved_model.signature_def_utils.build_signature_def(
        inputs={'input': tensor_info_input},
        outputs={'prediction': tensor_info_output},
        method_name=signature_constants.PREDICT_METHOD_NAME)
)

## Export the model for TensorFlow-Serving

In [4]:
model_version = "1"
export_path = os.path.join("../6-models/fruits", model_version)
tf_builder = builder.SavedModelBuilder(export_path)
with keras.backend.get_session() as sess:
    tf_builder.add_meta_graph_and_variables(
        sess=sess,
        tags=[tag_constants.SERVING],
        signature_def_map={
            signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: prediction_signature
            }
    )
    tf_builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b'../6-models/fruits/1/saved_model.pb'
